In [1]:
# 필요한 라이브러리 로드
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, StandardScaler
import optuna
from sklearn.model_selection import KFold

# 데이터 로드
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

# 배터리용량 결측치 대체 함수 정의
def fill_battery_capacity(df):
    # 결측치 행만 처리
    missing_indices = df[df['배터리용량'].isnull()].index

    # 차량상태가 Brand New인 경우
    brand_new_avg = df[df['차량상태'] == 'Brand New']['배터리용량'].mean()

    for idx in missing_indices:
        row = df.loc[idx]

        if row['차량상태'] == 'Brand New':
            # 1. Brand New: 차량 상태 평균으로 대체
            df.loc[idx, '배터리용량'] = brand_new_avg
        elif row['모델'] == 'IONIQ':
            # 3. 모델이 IONIQ: 같은 차량 상태 + 보증기간 평균으로 대체
            same_condition_warranty = df[(df['차량상태'] == row['차량상태']) & 
                                         (df['보증기간(년)'] == row['보증기간(년)'])]['배터리용량']
            if not same_condition_warranty.empty:
                df.loc[idx, '배터리용량'] = same_condition_warranty.mean()
        else:
            # 2. 그 외: 같은 차량 상태 + 모델 평균으로 대체
            same_condition_model = df[(df['차량상태'] == row['차량상태']) & 
                                       (df['모델'] == row['모델'])]['배터리용량']
            if not same_condition_model.empty:
                df.loc[idx, '배터리용량'] = same_condition_model.mean()

    return df

# 결측치 처리
train = fill_battery_capacity(train)
test = fill_battery_capacity(test)

# 결측치가 모두 처리되었는지 확인
print(f"Train 결측치: {train['배터리용량'].isnull().sum()}")
print(f"Test 결측치: {test['배터리용량'].isnull().sum()}")


# 사고이력과 차량상태를 더미 변수로 변환
train = pd.get_dummies(train, columns=['제조사', '모델', '구동방식'], drop_first=True)
test = pd.get_dummies(test, columns=['제조사', '모델', '구동방식'], drop_first=True)

# 차량상태 값을 순서를 고려해 직접 매핑
condition_mapping = {
    'Pre-Owned': 0,
    'Nearly New': 1,
    'Brand New': 2
}

# 차량상태 컬럼에 매핑 적용
train['차량상태'] = train['차량상태'].map(condition_mapping)
test['차량상태'] = test['차량상태'].map(condition_mapping)

# Label Encoding for '차량상태'
label_encoder = LabelEncoder()
train['사고이력'] = label_encoder.fit_transform(train['사고이력'])
test['사고이력'] = label_encoder.transform(test['사고이력'])

# 파생변수 생성
train['배터리효율'] = train['배터리용량'] / (train['주행거리(km)'] + 1)  # 주행거리가 0일 경우를 방지
test['배터리효율'] = test['배터리용량'] / (test['주행거리(km)'] + 1)


# # 수치형 변수
# continuous_vars = ['보증기간(년)', '연식(년)', '주행거리(km)', '배터리용량', '배터리효율']
# 
# 
# # Initialize the scaler
# scaler = StandardScaler()
# 
# # Fit and transform the scaler on train data, and transform the test data
# train[continuous_vars] = scaler.fit_transform(train[continuous_vars])
# test[continuous_vars] = scaler.transform(test[continuous_vars])

# 타깃 변수와 특성 분리
X = train.drop(columns=['ID', '가격(백만원)'])
y = train['가격(백만원)']
test_data = test.drop(columns=['ID'])

print("특성 개수", len(X.columns))
print("특성", X.info())

from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(df):
    """
    Calculate Variance Inflation Factor (VIF) for a DataFrame.

    Parameters:
        df (DataFrame): DataFrame containing numerical features for VIF calculation.

    Returns:
        DataFrame: VIF values for each feature.
    """
    # Ensure only numeric columns are used
    numeric_df = df.select_dtypes(include=['number'])

    # Drop rows with missing values to avoid errors
    numeric_df = numeric_df.dropna()

    vif_data = pd.DataFrame()
    vif_data["Feature"] = numeric_df.columns
    vif_data["VIF"] = [
        variance_inflation_factor(numeric_df.values, i) for i in range(numeric_df.shape[1])
    ]
    return vif_data

# Example usage:
# Assuming 'X' is your DataFrame with both numeric and non-numeric columns
vif_results = calculate_vif(X)
print(vif_results)

# X = X.drop(columns=['배터리용량'])
# test_data = test_data.drop(columns=['배터리용량'])
# 
# vif_results = calculate_vif(X)
# print(vif_results)
# 
# X = X.drop(columns=['보증기간(년)'])
# test_data = test_data.drop(columns=['보증기간(년)'])
# 
# vif_results = calculate_vif(X)
# print(vif_results)

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import ElasticNet
# from sklearn.model_selection import cross_val_score
# from xgboost import XGBRegressor
# from catboost import CatBoostRegressor
# 
# # 모델 리스트
# models = {
#     'Random Forest': RandomForestRegressor(random_state=42),
#     'ElasticNet': ElasticNet(random_state=42),
#     'XGBoost': XGBRegressor(random_state=42),
#     'CatBoost': CatBoostRegressor(verbose=0, random_state=42),
#     'LightGBM': lgb.LGBMRegressor(random_state=42)
# }
# 
# # 결과 저장
# cv_results = {}
# 
# # 5-Fold Cross-Validation 수행
# for model_name, model in models.items():
#     print(f"Training {model_name}...")
#     scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')  # RMSE로 평가
#     cv_results[model_name] = -scores.mean()  # 평균 RMSE 저장
# 
# # 결과 출력
# for model_name, rmse in cv_results.items():
#     print(f"{model_name}: Mean RMSE = {rmse:.4f}")



# LightGBM, XGBoost, CatBoost Optuna 최적화
def objective(trial, model_type):
    if model_type == 'lightgbm':
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': trial.suggest_float('learning_rate', 0.0005, 0.1, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'random_state': 42
        }
        lgb_dataset = lgb.Dataset(X, y)
        cv_results = lgb.cv(
            params,
            lgb_dataset,
            num_boost_round=500,
            nfold=5,
            metrics='rmse',
            stratified=False,
            seed=42        )
        return cv_results['valid rmse-mean'][-1]


# 모델별 Optuna 최적화
lgb_study = optuna.create_study(direction='minimize')
lgb_study.optimize(lambda trial: objective(trial, 'lightgbm'), n_trials=30)
lgb_best_params = lgb_study.best_params
lgb_best_rmse = lgb_study.best_value


# 최적 모델 선택
best_model_type = min(
    [('lightgbm', lgb_best_rmse, lgb_best_params)],
    key=lambda x: x[1]
)

print(f"Best Model: {best_model_type[0]} with RMSE: {best_model_type[1]}")

# 최적 모델 학습 및 예측
if best_model_type[0] == 'lightgbm':
    final_model = lgb.train(
        {**best_model_type[2], 'objective': 'regression', 'metric': 'rmse'},
        lgb.Dataset(X, y),
        num_boost_round=500
    )
    final_pred = final_model.predict(test_data)


# 제출 파일 생성
submission['가격(백만원)'] = final_pred
submission.to_csv('submission.csv', index=False)
print("제출 파일이 'submission.csv'로 저장되었습니다.")

C:\Users\kms10\anaconda3\envs\py3_12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train 결측치: 0
Test 결측치: 0
특성 개수 35
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   차량상태       7497 non-null   int64  
 1   배터리용량      7497 non-null   float64
 2   주행거리(km)   7497 non-null   int64  
 3   보증기간(년)    7497 non-null   int64  
 4   사고이력       7497 non-null   int32  
 5   연식(년)      7497 non-null   int64  
 6   제조사_B사     7497 non-null   bool   
 7   제조사_H사     7497 non-null   bool   
 8   제조사_K사     7497 non-null   bool   
 9   제조사_P사     7497 non-null   bool   
 10  제조사_T사     7497 non-null   bool   
 11  제조사_V사     7497 non-null   bool   
 12  모델_ID4     7497 non-null   bool   
 13  모델_ION5    7497 non-null   bool   
 14  모델_ION6    7497 non-null   bool   
 15  모델_IONIQ   7497 non-null   bool   
 16  모델_KNE     7497 non-null   bool   
 17  모델_M3      7497 non-null   bool   
 18  모델_MS      7497 non-null   bool   
 19  모델_MX      749

[I 2025-01-13 23:24:14,425] A new study created in memory with name: no-name-9f9c4219-738f-4acb-852c-273813c5a521


    Feature        VIF
0      차량상태  30.099766
1     배터리용량  32.862497
2  주행거리(km)   6.031782
3   보증기간(년)   8.695961
4      사고이력   1.046548
5     연식(년)   1.256149
6     배터리효율   1.016249
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=20) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1048576) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=20) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1048576) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=20) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1048576

[I 2025-01-13 23:24:25,771] Trial 0 finished with value: 1.4830669283908828 and parameters: {'learning_rate': 0.09842485407935744, 'subsample': 0.9472238224301078, 'colsample_bytree': 0.5098602234540528, 'max_depth': 20}. Best is trial 0 with value: 1.4830669283908828.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=11) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=2048) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=11) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=2048) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=11) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=2048) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [I

[I 2025-01-13 23:33:47,384] Trial 28 finished with value: 9.077230231386618 and parameters: {'learning_rate': 0.001455866609787069, 'subsample': 0.6682622416594904, 'colsample_bytree': 0.7799195244843309, 'max_depth': 11}. Best is trial 15 with value: 1.3518639837819648.


[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=19) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=524288) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=19) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=524288) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=19) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=524288) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing 

[I 2025-01-13 23:33:56,497] Trial 29 finished with value: 1.386032801809341 and parameters: {'learning_rate': 0.015461634160710985, 'subsample': 0.7278768221983536, 'colsample_bytree': 0.50563248789591, 'max_depth': 19}. Best is trial 15 with value: 1.3518639837819648.


Best Model: lightgbm with RMSE: 1.3518639837819648
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=11) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=2048) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=11) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=2048) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000562 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 704
[LightGBM] [Info] Number of data points in the train set: 7497, number of used fea